In [1]:
import os
import numpy as np
import pandas as pd
import swifter
import utils as utils
import catboost

In [2]:
DATA_PATH = "../IDAO-MuID"

In [3]:
test = utils.load_full_private_test_csv(DATA_PATH)

/home/ml/rumyantsev/venv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
closest_hits_features = test.swifter.apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)

Pandas Apply: 100%|██████████| 1452188/1452188 [15:27<00:00, 1565.99it/s]


In [7]:
test_concat = pd.concat(
    [test.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features],
    axis=1)

In [8]:
model = catboost.CatBoostClassifier().load_model("track_1_model_2500.cbm")
predictions = model.predict(test_concat.values, prediction_type="RawFormulaVal").astype(np.float32)
prediction_pd = pd.DataFrame(data={"prediction": predictions}, index=test.index)

In [9]:
prediction_pd.to_csv('submission_private.csv')